In [111]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
from pprint import pprint

from pprint import pprint

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score

In [112]:
weather=pd.read_csv('./data/temps_extended.csv')
y = weather['actual']
X = weather.drop(['actual','weekday','month','day','year'],axis =1)
X_train, X_val, Y_train,Y_val=train_test_split(X,y,test_size=0.3, random_state=42)

In [113]:
rf = RandomForestRegressor(random_state = 42)

# Look at parameters used by our current forest
print('Параметры по умолчанию:\n')
pprint(rf.get_params())

Параметры по умолчанию:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [114]:

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt'] # выбирать все признаки или число равное корню из количества признаков

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

min_samples_split = [2, 5, 10] 
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators, # Количество деревьев в лесу
               'max_features': max_features, # Количество признаков максимально или число равное корню из максимума
               'max_depth': max_depth, # максимальная глубина дерева
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap} # применять бутстрап или нет
               

In [115]:
random_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

# Обучим лес

n_iter - количество итераций изменений выбранных настроек параметров. n_iter делает ставку на время выполнения и качество решения.

cv - это сколько будет кросс валидаций для каждой модели

verbose контролирует подобие

random_state - состояние генератора псевдослучайных чисел, используемое для случайной однородной выборки из списков возможных значений

n_jobs - управляет количеством заданий, отправляемых при параллельном выполнении. Уменьшение этого числа может быть полезно, чтобы избежать резкого увеличения потребления памяти, когда отправляется больше заданий, чем может обработать ЦП.

In [116]:
rf = RandomForestRegressor(random_state=42)
rf_random = RandomizedSearchCV(estimator = rf, 
param_distributions = random_grid,
n_iter = 100,
cv = 3, 
verbose=2,
random_state=42,
n_jobs = -1
    )
rf_random.fit(X_train, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  2.4min finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [117]:
optimal_grid = rf_random.best_params_
optimal_grid

{'n_estimators': 1000,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': True}

In [118]:
y_pred1 = rf_random.predict(X_val)

In [127]:
 optimal_grid = {'n_estimators': [1000],
 'min_samples_split': [5],
 'min_samples_leaf': [2],
 'max_features': ['sqrt'],
 'max_depth': [10],
 'bootstrap': [True]}

In [133]:
rf2 = RandomForestRegressor(random_state=42)
rf_random2 = RandomizedSearchCV(estimator = rf2, 
param_distributions = optimal_grid,
n_iter = 100,
cv = 3, 
verbose=2,
random_state=42,
n_jobs = -1
    )
rf_random2.fit(X_train, Y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    4.0s finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True], 'max_depth': [10],
                                        'max_features': ['sqrt'],
                                        'min_samples_leaf': [2],
                                        'min_samples_split': [5],
                                        'n_estimators': [1000]},
                   random_state=42, verbose=2)

In [134]:
y_pred2 = rf_random2.predict(X_val)

In [135]:
mse1 = mean_squared_error(Y_val,y_pred1)
mse1

23.10053911088241

In [136]:
mse2 = mean_squared_error(Y_val,y_pred2)
mse2

23.10053911088241

In [137]:
mse2 - mse1

0.0